In [93]:
# email2inventory
# read evergreen email
# this script will fail if scripts/email2inventory/data/evergreen-20200930.txt doesn't exist

import os
import re

def evergreen_read(filename):
    date = ""
    product_flag = False
    position = 0
    items = {}
    f = open(filename)
    lines = f.readlines()
    while not product_flag:
        line = lines.pop(0).strip()
        m = re.match(r"Date: (\w* \d+, \d{4})", line)
        if m:
            date = m.group(1)
        if line == "Product":
            product_flag = True
    line = lines.pop(0).strip()
    assert line == "Quantity"
    line = lines.pop(0).strip()
    assert line == "Price"
    while product_flag:
        m = re.match(r"(.*) - 1 Week", lines.pop(0).strip())
        try:
            itemname = m.group(1)
        except:
            break
        quantity = lines.pop(0).strip()
        price = lines.pop(0).strip()
        items[itemname] = [quantity,price]
    f.close()
    return({'items': items, 'date' : date})
    
print(evergreen_read(os.path.join("email_data/evergreen-20200930.txt")))

{'items': {"Red's Best Hake": ['2', '$32.00']}, 'date': 'September 30, 2020'}


In [166]:
# email2inventory
# read kistler email
# this script will fail if scripts/email2inventory/data/kistler-20200912.txt doesn't exist

import os
import re
import requests
import datetime

client_secret = subprocess.check_output("./sortly_key.sh").strip()
post_url = 'https://api.sortly.co/api/v1/items'

def kistler_read(filename):
    date = ""
    product_flag = False
    position = 0
    items = {}
    f = open(filename)
    lines = f.readlines()
    while not product_flag and lines:
        line = lines.pop(0).strip()
# grabbing the date from the email header
        m = re.match(r"Date: (\w* \d+, \d{4})", line)
        if m:
            date = datetime.datetime.strptime(m.group(1), '%B %d, %Y')
            iso_date = str(date_time_obj).replace('+00:00', 'Z')
# this version still assumes that there is a single item which is a several sets of wines
# in a bundle - need to generalize
        elif line == "Total":
            product_flag = True
        while product_flag:
            m = re.search(r"(\d+)-Bottle Mixed Package\*$", lines.pop(0).strip())
            items = get_package(int(m.group(1)),lines)
            product_flag = False
    f.close()
    return({'items' : items, 'iso_date' : iso_date})

def get_package(bottles,lines):
    inventory = {}
    while bottles > 0:
        line = lines.pop(0).strip()
        m = re.match(r"\•(\d+) btls (\d{4}) (.*)$",line)
        inventory[m.group(3)] = [int(m.group(1)), m.group(2)]
        bottles -= int(m.group(1))
    return inventory
    
inventory = kistler_read(os.path.join("email_data/kistler-20200912.txt"))

for item in inventory['items']:
#    print(item)
#    print(inventory['items'][item])
#    print(inventory['items'][item][0])
# posting to Sortly API
    post_url = 'https://api.sortly.co/api/v1/items'
    data = {'name' : item, 'quantity' : inventory['items'][item][0], 'type' : 'item', 'tags' : [{'name' : 'test'}, {'name' : 'wine'}], 'custom_attribute_values': [{'value': inventory['iso_date'], 'custom_attribute_id': 170931, 'custom_attribute_name': 'Purchase Date'}]}
    response = requests.post(post_url, json=data, headers=headers)

McCrea Vineyard Chardonnay Sonoma Mountain
[2, '2018']
2
Durell Vineyard Chardonnay Sonoma Coast
[2, '2018']
2
Laguna Ridge Vineyard Pinot Noir Russian River Valley
[2, '2018']
2


This uses OAUTH2 to get the two folders from the Sortly API and then open an item and get the image.

Boris figured out how to do it using `urllib.request`, but I think I should be able to do it using `requests`.  Also, I think we're doing some sort of brute force OAUTH and that there is something more complicated but proper - maybe using requests. Something to try to figure out.

I moved the client key to sortly_key.sh as a file that prints the key. I ignored it in GitHub and am not syncing it.

Now trying to generalize for the farmers to you email

In [17]:
# email2inventory
# read farmers to you email
# this script will fail if scripts/email2inventory/data/f2u-20201006.txt doesn't exist

import os
import re
import requests
import datetime
import subprocess
import urllib.request
import json
import io
import subprocess

client_secret = subprocess.check_output("./sortly_key.sh").strip()
working_path = os.getenv("HOME") + '/GitHub/joi.github.io/__scripts/'
post_url = 'https://api.sortly.co/api/v1/items'
product_pattern = 'Item'
pop_fields = ['Item', 'Wt', '$/lb', 'Qty', 'Cost']
sortly_fields = ['name', 'notes', '', 'quantity', 'price']
end_items = ""
# data_file = 'email_data/f2u-20201006.txt'
data_file = 'email_data/f2u-oneitem.txt'


def parse_file(file_str):
    date = ""
    product_flag = False
    position = 0
    items = {}
    
#    print("opening file" + data_file)
#    f = open(working_path + filename)
    lines = file_str.splitlines()
    
    while not product_flag and position < len(lines):
        line = lines[position].strip()
        position += 1
# grabbing the date from the email header
# date format for gmail is: 2020106 16:14:19 GMT-4
#        m = re.match(r"^(\d{4}) [\d:]{8}", line)
#        if m:
#            date_time_obj = datetime.datetime.strptime(m.group(1), '%Y%m%d')
#            iso_date = str(date_time_obj.isoformat()).replace('00:00:00', '00:00:00.000Z')
#            print("date: " + iso_date)
# if product_pattern matches a line, it means that the part of the email with the item
# list has begun so start reading the products
        product_pattern = "\t".join(pop_fields)
#        print("product pattern " + product_pattern)
        if line == product_pattern:
            product_flag = True
            position += 1
            line = lines[position].strip()
            item_fields = line.split("\t")
#            print(item_fields)
        while product_flag:
# now read in the actual product data
            for i in range(len(item_fields)):
                current_field = sortly_fields[i]
                current_item = item_fields[i].strip()
                print(current_item)
                if current_field =='name':
                    name = current_item
#                    print("found name: " + name)
                    if name not in items:
                        items[name] = {}
# if there is more than one of the same item name, add a number at the end of the name and increment it
# each time
                    else:
                        j = 1
                        while name + " " + str(j) in items:
                            j += 1
                        name = name + " " + str(j)
                        items[name] = {}
                elif current_field != '':
 #                   print("name: " + name)
                    if current_field == 'price':
 #                       print("price: " + current_item[1:])
                        items[name]['price'] = float(current_item[1:])
                    elif current_field == 'quantity':
                        items[name]['quantity'] = int(current_item)
                    else:
                        items[name][current_field] = item_fields[i]
            position += 1
            print("position: " + str(position))
            line = lines[position].strip()
            item_fields = line.split("\t")
            print(line)
            if line == end_items:
                product_flag = False
    return items

def get_package(bottles,lines):
# for getting bottles inside of a Kistler mixed set of bottles
    inventory = {}
    while bottles > 0:
        line = lines.pop(0).strip()
        m = re.match(r"\•(\d+) btls (\d{4}) (.*)$",line)
        inventory[m.group(3)] = [int(m.group(1)), m.group(2)]
        bottles -= int(m.group(1))
    return inventory

def post_shortly(inventory, date):
# sortly POST API URL
    url = 'https://api.sortly.co/api/v1/items'
# get my secret key
    client_secret = subprocess.check_output("./sortly_key.sh").strip()
    headers = {'Accept': 'application/json','Authorization': client_secret}
    data = {}
    for item in inventory:
        data = {'name' : str(item)}
        for key, value in inventory[item].items():
            data[key] = value
        data['type'] = 'item'
        data['tags'] = [{'name' : 'pending'},{'name' : 'test'}]
        data['custom_attribute_values'] = [{'value' : date, 'custom_attribute_id': 170931, 'custom_attribute_name': 'Purchase Date' },{'value': None, 'custom_attribute_id': 171873, 'custom_attribute_name': 'Product URL'}]
        print("posting " + str(item))
# add the item to sortly
        response = requests.post(url, json=data, headers=headers)
    return response

# old main section
# inventory, date = get_file(data_file)
# post_shortly(inventory, date)

# get email of inventory

def GetMimeMessage(service, user_id, msg_id):
    message = service.users().messages().get(userId=user_id, id=msg_id, format='raw').execute()
    msg_str = base64.urlsafe_b64decode(message['raw'].replace('-_', '+/').encode('ASCII')).decode("utf-8")
    mime_msg = email.message_from_string(msg_str)
    messageMainType = mime_msg.get_content_maintype()
    if messageMainType == 'multipart':
            for part in mime_msg.get_payload():
                if part.get_content_maintype() == 'text':
                    return part.get_payload()
            return ""
    elif messageMainType == 'text':
        return message

def gmail_auth():
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)
    return service

    # Call the Gmail API
def find_message_id(service, subject_pattern):
    results = service.users().messages().list(userId='me', q=subject_pattern).execute()
    messages = results.get('messages', [])
    if not messages:
        print('No messages found.')
    else:
        print('Messages:')
        for message in messages:
            messageheader = service.users().messages().get(userId='me',id=message['id'],format="full").execute()
            headers=messageheader["payload"]["headers"]
            print("message ID: " + str(message['id']))
            subject= [i['value'] for i in headers if i["name"]=="Subject"]
            date = [i['value'] for i in headers if i["name"]=="Date"]
            print(subject)
            print(date)
            return(message['id'], date)
        
def clean_message(message):
    msg_cln = re.sub(r'=[A-F0-9]{2}', '', message)
    msg_cln =  re.sub(r'^> ', '', msg_cln, flags=re.MULTILINE)
    msg_cln =  re.sub(r'^>', '', msg_cln, flags=re.MULTILINE)
    msg_cln =  re.sub(r'=\r\n', '', msg_cln, flags=re.MULTILINE)
    msg_cln =  re.sub(r'^: ', '', msg_cln, flags=re.MULTILINE)
    msg_cln =  re.sub(r'^:', '', msg_cln, flags=re.MULTILINE)
    return msg_cln
    pass

def convertgmaildate2iso(date):
    date_time_obj = datetime.datetime.strptime(date[0], '%a, %d %b %Y %H:%M:%S %z')
    iso_date = str(date_time_obj.isoformat()).replace('00:00:00', '00:00:00.000Z')
    return iso_date

subject_pattern = '\"Your Farmers To You Order This Week\"'
service = gmail_auth()
message_id, date = find_message_id(service, subject_pattern)
message = GetMimeMessage(service, 'me', message_id)
msg_cln = clean_message(message)
items = parse_file(msg_cln)
iso_date = convertgmaildate2iso(date)
print(items)
post_shortly(items, iso_date)
# message_list = clean_message(message)




Messages:
message ID: 174ffb29996bc281
['Fwd: Your Farmers To You Order This Week, Wednesday October 7th']
['Tue, 6 Oct 2020 16:55:24 -0400']
Milk - Creamline Organic - Strfrd
-
-
1
$6.40
position: 31
Buttermilk - Organic	-	-	1	$3.90
Buttermilk - Organic
-
-
1
$3.90
position: 32
Cream - Heavy Organic	-	-	1	$7.90
Cream - Heavy Organic
-
-
1
$7.90
position: 33
Cheese - Melting Blend	-	-	1	$11.90
Cheese - Melting Blend
-
-
1
$11.90
position: 34
Cheese - Triple Cream Organic	-	-	1	$6.40
Cheese - Triple Cream Organic
-
-
1
$6.40
position: 35
Cheese - Bayley Hazen Blue	-	-	1	$12.90
Cheese - Bayley Hazen Blue
-
-
1
$12.90
position: 36
Cheese - Goat Feta	-	-	1	$11.80
Cheese - Goat Feta
-
-
1
$11.80
position: 37
Raspberries - Organic	-	-	1	$6.90
Raspberries - Organic
-
-
1
$6.90
position: 38
Butter - Cultured Unsalted	-	-	2	$15.60
Butter - Cultured Unsalted
-
-
2
$15.60
position: 39
Eggs - Ungraded GMO Free - OF	-	-	1	$7.90
Eggs - Ungraded GMO Free - OF
-
-
1
$7.90
position: 40
Sauce - Pizza - 

posting Milk - Creamline Organic - Strfrd
posting Buttermilk - Organic
posting Cream - Heavy Organic
posting Cheese - Melting Blend
posting Cheese - Triple Cream Organic
posting Cheese - Bayley Hazen Blue
posting Cheese - Goat Feta
posting Raspberries - Organic
posting Butter - Cultured Unsalted
posting Eggs - Ungraded GMO Free - OF
posting Sauce - Pizza - VO
posting Vinegar - Maple Balsamic
posting Granola - 22 oz BIG bag - Nutty Stephs
posting Grains - Ancient Grains Cereal - Organic
posting Dried Cranberries - Organic
posting Beef - Chuck Roast
posting Beef - Short Ribs
posting Beef - Short Ribs 1
posting Beef - Short Ribs 2
posting Beef - Short Ribs 3
posting Beef - Tongue
posting Chicken -Tenders - Non GMO
posting Pork - Spare Ribs
posting Pork - Spare Ribs 1
posting Pork - Spare Ribs 2
posting Pork - Spare Ribs 3
posting Pork -Boston Butt - Bone In
posting Pork -Boston Butt - Bone In 1
posting Beef - Ground Beef - G&T
posting Pork Sausage -Sweet Italian - SVF
posting Eggs - Ungra

<Response [200]>

### To Do
Various things to do:
- ~~~IMAP integration to get mail~~~
- List of source and formats
- Trigger when there is a new email

In [1]:
# gmail/quickstart/quickstart.py

from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
# from https://stackoverflow.com/questions/50630130/how-to-retrieve-the-whole-message-body-using-gmail-api-python
# take a look at this: https://www.thepythoncode.com/article/reading-emails-in-python
import base64
import email
from apiclient import errors

def GetMessage(service, user_id, msg_id):
    message = service.users().messages().get(userId=user_id, id=msg_id).execute()
    print('Message snippet:' + message['snippet'])
    return message

def GetMimeMessage(service, user_id, msg_id):
    message = service.users().messages().get(userId=user_id, id=msg_id, format='raw').execute()
    msg_str = base64.urlsafe_b64decode(message['raw'].replace('-_', '+/').encode('ASCII')).decode("utf-8")
    mime_msg = email.message_from_string(msg_str)
    messageMainType = mime_msg.get_content_maintype()
    if messageMainType == 'multipart':
            for part in mime_msg.get_payload():
                if part.get_content_maintype() == 'text':
                    return part.get_payload()
            return ""
    elif messageMainType == 'text':
        return message


def main():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)

    # Call the Gmail API

    results = service.users().messages().list(userId='me', q='\"Your Farmers To You Order This Week\"').execute()
    messages = results.get('messages', [])

    if not messages:
        print('No messages found.')
    else:
        print('Messages:')
        for message in messages:
            messageheader = service.users().messages().get(userId='me',id=message['id'],format="full").execute()
            headers=messageheader["payload"]["headers"]
            print("message ID: " + str(message['id']))
            subject= [i['value'] for i in headers if i["name"]=="Subject"]
            print(subject)
            print(GetMimeMessage(service, 'joiito@gmail.com', message['id']))

if __name__ == '__main__':
    main()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=647405849799-tt6hhqo7qh68cgnn6kifgvq36qm64bvu.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A56497%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=0Lsw4tW5fFGjwxTho8FWhxoL6ymUch&access_type=offline
Messages:
message ID: 174ffb29996bc281
['Fwd: Your Farmers To You Order This Week, Wednesday October 7th']



=E8=BB=A2=E9=80=81=E3=81=95=E3=82=8C=E3=81=9F=E3=83=A1=E3=83=83=E3=82=BB=E3=83=
=BC=E3=82=B8:

> =E5=B7=AE=E5=87=BA=E4=BA=BA: "FarmersToYou.com Orders" <info@farmerstoyou.=
com>
> =E6=97=A5=E6=99=82: 2020=E5=B9=B410=E6=9C=886=E6=97=A5 16:14:19 GMT-4
> =E5=AE=9B=E5=85=88: Mizuka Ito <mizuka@ito.com>
> =E4=BB=B6=E5=90=8D: Your Farmers To You Order This Week, Wednesday October=
 7th
>=20
> =EF=BB=BF
> Feed your family and your values.
>=20
> =20
> Dear Mizuka,
>=20
> Thank you for partnering with Farmers to You. Your order 

In [ ]:
# stashing this code for now
def try_sorty()
# trying to access Sortly API
import urllib.request
import json
import requests
import io
import subprocess
from PIL import Image

client_secret = subprocess.check_output("./sortly_key.sh").strip()

headers = {
  'Accept': 'application/json',
  'Authorization': client_secret
}
response = requests.get('https://api.sortly.co/api/v1/items?per_page=&page=&folder_id=&include=', headers=headers).json()
print("Folder: " + response['data'][0]['name'])
next_url = response['meta']['next_page_url']
response = requests.get(next_url, headers=headers).json()
print(response)
print("Folder: " + response['data'][0]['name'])
next_url = response['meta']['next_page_url']
response = requests.get('https://api.sortly.co/api/v1/items/17481596?include=custom_attributes%2Cphotos', headers=headers).json()
photo_url = response['data']['photos'][0]['url']
print(response['data']['custom_attribute_values'])
response = requests.get(photo_url)
img = Image.open(io.BytesIO(response.content))
print("here's an item with a photo")
img.show()
# Posting test object
url = 'https://api.sortly.co/api/v1/items'

data = {'name' : "test object", 'type' : 'item', 'tags' : [{'name' : 'test'}]}
# print(url, data, headers)
response = requests.post(url, json=data, headers=headers)

In [118]:
# Unit Tests
import unittest

class TestNotebook(unittest.TestCase):
    
    def test_get_file_date(self):
        # check date formatr
        inventory, date = get_file(data_file)
        self.assertTrue(re.match(r'\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\.000Z', date))

unittest.main(argv=[''], verbosity=2, exit=False)

test_get_file_date (__main__.TestNotebook) ... 

opening fileemail_data/f2u-oneitem.txt


ok

----------------------------------------------------------------------
Ran 1 test in 0.002s

OK
